# 4.1. What is Ownership

Ownership is a set of rules that govern how a Rust program manages memory.

## The stack and Heap

Both the stack and the heap are parts of memory available to your code to use at runtime, but they are structured in different ways.

- Stack
    - FILO, LIFO
    - Adding or removing elements from the middle or bottom won't work as well.
    - push, pop
    - All data on stack must have **known, fixed size**
 
- Heap
    - Request a certain amount of space to store data.
    - Behavior : find bit rnough mem, mark it as 'in use', return the `pointer` (address)
 
- Comp
    - Pushing to the stack is faster than allocating on the heap.
    - Accessing data in the heap is slower than on stack.
 
- Function
  Then the code calls a function, the values passed into the function and function's local variables get pushed onto the stack.


## Ownership Rules

- Each value in Rust has an owner.
- There can only be one owner at a time.
- When the owner goes out of scope, the value will be dropped.

## Variable Scope
A scope is the range within a program for which an item is valid.

In [2]:
{
    {
        let s = "hello";
        println!("s = {}", s);
    }

    println!("s = {}", s); // the scope of s is over, s is invalid here.
}

Error: cannot find value `s` in this scope

## The String Type

The `String` type is a good example to illustrate the rules of ownership other than the basic scalar data types.

- `string` literals are immutable
- To satisfy the situations that we need mutable string, Rust has a second string type `String`

In [6]:
{
    let mut s = String::from("Hello");

    s.push_str(", World!");

    println!("{}", s);
}

Hello, World!


()

### Memory and Allocation

In the case of string literal, the contents are known at compile time, that's why  string literals are fast and efficient.

With the `String` type, in order to support a mutable, growable piece of text

1. The memory must be requested from the memory allocator at runtime.
2. We need a way of returning this memory to the allocator when we're done with our `String`
 
The first part is done by user: when we call `String::from`, its implementation requests the memory it needs.

The second part (returning the memory, explicitly call `free`?) is different in Rust: 

The memory is **automatically returned once the variable that owns it goes out of scope**.

As the below example, when `s` goes out of scope, Rust automatically calls a special function for us. The function is called `drop`, and it's where the author of `String` can put the code to return the memory.


In [7]:
{
    {
        let s = "hello";
        println!("s = {}", s);
    }

    println!("s = {}", s); // the scope of s is over, s is invalid here.
}

Error: cannot find value `s` in this scope

### Variables and Data Interacting with Move

Multiple variables can interact with the same data in different ways.

In the below example: "bind the value `5` to `x`; make a copy of the `x` and bind it to `y`"

In [8]:
{
    let x = 5;
    let y = x;

    println!("x = {}, y = {}", x, y)
}

x = 5, y = 5


()

It will be different in `String` case.

A `String` is made up of 3 parts:
- ptr to the memory
- length
- capacity

<img src = "../assets/0x4_001.jpg" width=400>

When we assign `s1` to `s2`, the `String` data is copied (ptr, len, capacity), the real content was not.

<img src = "../assets/0x4_002.jpg" width=400>

But in Rust, it will call the `drop` function to clean up the memory when a variable goes out of scope, here is the conflict.

When `s2` and `s1` go out of scope, they will both try to free the same memory. This is known as a double free error and is one of the memory safety bugs we mentioned previously. 

In [10]:
{
    let s1 = String::from("Hello");
    let s2 = s1;

    println!("s1 = {}, s2 = {}", s1, s2);
}

Error: borrow of moved value: `s1`

### Variable and Data Interacting with Clone

If we want to deeply copy the heap data of the `String`, `clone` is the method we could use.

In [11]:
{
    let s1 = String::from("Hello");
    let s2 = s1.clone();

    println!("s1 = {}, s2 = {}", s1, s2);
}

s1 = Hello, s2 = Hello


()

### Stack-only Data: Copy

When looking back to the integer example, here is no such memory conflict issue.

The reason is that stack data has the below features, which make the copy of the data very fast.
- has known size at compile time
- stored entirely on the stack

Rust has a special annotation called the `Copy` trait that we can place on types that are stored on the stack. If a type implements the `Copy` trait, variable that use it do not move but copied (valid as integer demo).

Types implement the `Copy` trait:

- All the integer
- The Boolean type
- All the floating-point type
- The character type
- Tuples, if they only contain types with `Copy` trait implemented.

## Ownership and Functions

The mechanics of passing a value to a function are similar to assigning a value to a variable (move or copy).

In [15]:
{
    fn call_fn() {
        let x : u32 = 30; // x comes to the scope
        makes_copy(x); // x was copied
        println!("x = {}", x); // valid x
        
        let s = String::from("Hello"); // s comes to the scope
        takes_ownership(s); // s was moved to func takes_ownership
        println!("s = {}", s); // invalid s
    }

    fn takes_ownership(some_str: String) {
        println!("{}", some_str);
    }

    fn makes_copy(some_int: u32) {
        println!("{}", some_int);
    }

    call_fn();
}

Error: borrow of moved value: `s`

# 4.2. References and Borrowing

A reference is like a pointer in that it’s an address we can follow to access the data stored at that address; that data is owned by some other variable.

Unlike a pointer, a reference is guaranteed to point to a valid value of a particular type for the life of that reference.

In the below example, these ampersands represent references, and they allow you to refer to some value without taking ownership of it.

<img src="../assets/0x4_003.jpg" width=500>

The action of creating a `reference` is called as `borowwing`.

In [18]:
{
    fn cal_len(s: &String) -> usize {  // s is a reference to a String
        s.len()
    } // s goes out of scope, but s has no ownership of the data, drop wasn't called here.

    let s1 = String::from("Hello");
    let len = cal_len(&s1);

    println!("string '{}' len is {}", s1, len);
}

string 'Hello' len is 5


()

### Mutable References

We need mutable reference to change the value of variable with `&mut`.

In [23]:
{
    fn change_val(s : &String) {
        s.push_str(", World!");
    }
    
    let mut s = String::from("Hello");

    change_val(&s);
    println!("{}", s);
}

Error: cannot borrow `*s` as mutable, as it is behind a `&` reference

In [21]:
{
    fn change_val(s : &mut String) {
        s.push_str(", World!");
    }
    
    let mut s = String::from("Hello");

    change_val(&mut s);
    println!("{}", s);
}

Hello, World!


()

But be noted : **if you have a mutable reference to a value, you can have no other reference to that value**

The benefit of having this restriction is that Rust can prevent data races at compile time, `data racing`:

- Two or more pointers access the same data at the same time
- At least one of the pointers is being used to write to the data
- There’s no mechanism being used to synchronize access to the data
ata.

In [26]:
{
    let mut s = String::from("Hello");

    let s1 = &mut s;
    let s2 = &mut s;

    println!("{}, {}", s1, s2);
}

Error: cannot borrow `s` as mutable more than once at a time

Rust enforces a similar rule for combining mutable and immutable references.

Users of an immutable reference don't expect the value to suddenly change out from under them.

In [28]:
{
    let mut s = String::from("Hello");

    let r1 = &s;
    let r2 = &s;
    let r3 = &mut s;

    println!("{}, {}, and {}", r1, r2, r3);
}

Error: cannot borrow `s` as mutable because it is also borrowed as immutable

In [29]:
{
    let mut s = String::from("hello");

    let r1 = &s; // no problem
    let r2 = &s; // no problem
    println!("{} and {}", r1, r2);
    // variables r1 and r2 will not be used after this point

    let r3 = &mut s; // no problem
    println!("{}", r3);

}

hello and hello
hello


()

## Dangling References

In Rust, the compiler guarantees that references will never be dangling references: if you have a reference to some data, the compiler will ensure that the data will not go out of scope before the reference to the data does.

In [31]:
{
    fn dangling() -> &String { // dangle returns a reference to a String
        let s = String::from("Hello"); // s is a new String

        &s // we return a reference to the String, s
    } // Here, s goes out of scope, and is dropped. Its memory goes away.
      // Danger!

    let r1 = dangling();
}

Error: missing lifetime specifier

- At any given time, you can have either one mutable reference or any number of immutable references.- 
References must always be valid.

# 4.3. The Slice Type

`Slices` let you reference a contiguous sequence of elements in a collection rather than the whole collection.

Assuming to solve a 'first_world' problem, we could use idx to mark it without slice type, but it gives some problem:

- The idx can only a meaningful number in the context of the `&String`
- The 'idx' will be complex to solve a bigger problem.

In [33]:
{
    fn first_word(s : &String) -> usize {
        let bytes = s.as_bytes();

        for (i, &item) in bytes.iter().enumerate() {
            if b' ' == item {
                return i;
            }
        }

        return s.len();
    }

    let mut s = String::from("Hello, World.");
    let word = first_word(&s);

    s.clear();

    println!("idx of first world {}", word);  // it could run, but the idx is invalid now as the String was clean up.
}

idx of first world 6


()

## String Slices

A `String slice` is a reference to part of a `String`.

<img src="../assets/0x4_004.jpg" width=400>

In [39]:
{
    let s = String::from("Hello, World!");

    let hello = &s[0..5];
    let world = &s[7..];
    let whole = &s[..];

    println!("{}, {} --> {}", hello, world, whole);
}

Hello, World! --> Hello, World!


()

In [41]:
{
    fn first_word(s : &String) -> &str {
        let bytes = s.as_bytes();

        for (i, &item) in bytes.iter().enumerate() {
            if b' ' == item {
                return &s[..i];
            }
        }

        return &s[..];
    }

    let mut s = String::from("Hello, World.");
    let word = first_word(&s);
    println!("The first word: {}", word);

    // s.clear();
    // println!("idx of first world {}", word);  // it could run, but the idx is invalid now as the String was clean up.
}

The first word: Hello,


()

In [42]:
{
    fn first_word(s : &String) -> &str {
        let bytes = s.as_bytes();

        for (i, &item) in bytes.iter().enumerate() {
            if b' ' == item {
                return &s[..i];
            }
        }

        return &s[..];
    }

    let mut s = String::from("Hello, World.");
    let word = first_word(&s);
    println!("The first word: {}", word);

    s.clear();
    println!("idx of first world {}", word);  // it could run, but the idx is invalid now as the String was clean up.
}

Error: cannot borrow `s` as mutable because it is also borrowed as immutable

## String Literal as Slices

In [43]:
{

    fn first_word(s : &str) -> &str {
        let bytes = s.as_bytes();

        for (i, &item) in bytes.iter().enumerate() {
            if b' ' == item {
                return &s[..i];
            }
        }

        return &s[..];
    }
    
    let my_string = String::from("hello world");

    // `first_word` works on slices of `String`s, whether partial or whole
    let word = first_word(&my_string[0..6]);
    let word = first_word(&my_string[..]);
    // `first_word` also works on references to `String`s, which are equivalent
    // to whole slices of `String`s
    let word = first_word(&my_string);

    let my_string_literal = "hello world";

    // `first_word` works on slices of string literals, whether partial or whole
    let word = first_word(&my_string_literal[0..6]);
    let word = first_word(&my_string_literal[..]);

    // Because string literals *are* string slices already,
    // this works too, without the slice syntax!
    let word = first_word(my_string_literal);
}

()

## Other Slices

In [44]:
{
    let a = [1, 2, 3, 4, 5];
    let slice = &a[1..3];
    
    assert_eq!(slice, &[2, 3]);

}

()

The concepts of ownership, borrowing, and slices ensure memory safety in Rust programs at compile time.

The Rust language gives you control over your memory usage in the same way as other systems programming languages, but having the owner of data automatically clean up that data when the owner goes out of scope means you don’t have to write and debug extra code to get this control.